In [25]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import sys

import numpy as np
import pandas as pd
import seaborn as sns
# testing
from pandas.testing import assert_frame_equal
from tqdm import tqdm

import j_utils.munging as mg
from lendingclub.lc_utils import gen_datasets
from lendingclub import config

pd.options.display.max_columns = 999
pd.options.display.max_rows = 60
pd.options.display.max_seq_items = None

# Make the train script

input should be model type (should try to accept list)
bundled with the parameters for each type of model

should return/create the saved model and anything necessary for data processing
for the model

In [27]:
%%writefile ../../lendingclub/modeling/08_train.py
import os
import sys
import argparse
import pickle
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from lendingclub import config, utils
import j_utils.munging as mg

def prepare_data(model_n, data, proc=None, ds_type='train'):
    '''
    returns the processed data for a model, which could be different between
    model types e.g. can handle categoricals or not. additionally returns
    a tuple of anything necessary to process valid/test data in the same manner
    ds_type must be 'train', 'valid', or 'test'
    '''
    assert ds_type in ['train', 'valid', 'test'], print('ds_type invalid')
    if model_n in ['baseline', 'A', 'B', 'C', 'D', 'E', 'F', 'G']:
        return data, None
#     elif model_n == 'logistic_regr':
    else:
        if ds_type == 'train':
            temp = mg.train_proc(data)
            procced = temp[0]
            return procced, temp[1:]
        elif ds_type in ['test', 'valid']:
            assert proc, print('must pass data processing artifacts')
            temp = mg.val_test_proc(data, *proc)
            return temp

    
def train_model(model_n, X_train, y_train, X_valid=None, y_valid=None):
    if model_n in ['baseline', 'A', 'B', 'C', 'D', 'E', 'F', 'G']:
        return 42
    elif model_n == 'logistic_regr':
        lr_model = LogisticRegression(class_weight='balanced')
        lr_model.fit(X_train, y_train)
        return lr_model
    
def export_models(m, model_n):
    if model_n in ['baseline', 'A', 'B', 'C', 'D', 'E', 'F', 'G']:
        with open(os.path.join(config.modeling_dir, '{0}_model.pkl'.format(model_n)), 'wb') as file:
            pickle.dump(m, file)
    elif model_n == 'logistic_regr':
        joblib.dump(m,os.path.join(config.modeling_dir, '{0}_model.pkl'.format(model_n)))
    
def export_data_processing(proc_arti, model_n):
    if model_n in ['baseline', 'A', 'B', 'C', 'D', 'E', 'F', 'G']:
        with open(os.path.join(config.modeling_dir, '{0}_model_proc_arti.pkl'.format(model_n)), 'wb') as file:
            pickle.dump(proc_arti, file)
    elif model_n == 'logistic_regr':
        joblib.dump(proc_arti, os.path.join(config.modeling_dir, '{0}_model_proc_arti.pkl'.format(model_n)))


parser = argparse.ArgumentParser()
parser.add_argument('--model', '-m', help='specify model(s) to train')

if not len(sys.argv) > 1:
    models = ['logistic_regr'] # , 'A', 'B', 'C', 'D', 'E', 'F', 'G'

args = parser.parse_args()
if args.model:
    models = args.model.split()
# models = ['logistic_regr']


if not os.path.isdir(config.modeling_dir):
    os.makedirs(config.modeling_dir)

tr_val_base_data, tr_val_eval_data, _ = utils.load_dataset(ds_type='train')
# ensure ordering is correct for time series split
tr_val_base_data, tr_val_eval_data = mg.sort_train_eval(tr_val_base_data, tr_val_eval_data, 'id', 'issue_d')


for model_n in models:
    # do 3 steps of TS cross validation, with valid size at 5% (20 splits)
    tscv = mg.time_series_data_split(tr_val_eval_data, 'issue_d', 20, 1)
    for tr_idx, val_idx in tscv:
        # split out validation from train_data
        y_train = tr_val_eval_data.loc[tr_idx, 'target_loose']
        y_valid = tr_val_eval_data.loc[val_idx, 'target_loose']
        X_train = tr_val_base_data.loc[tr_idx]
        X_valid = tr_val_base_data.loc[val_idx]
        
        X_train, proc_arti = prepare_data(model_n, X_train, ds_type='train')
        X_valid = prepare_data(model_n, X_valid, proc = proc_arti, ds_type='valid')
        m = train_model(model_n, X_train, y_train)

        #save stuff
        export_models(m, model_n)
        export_data_processing(proc_arti, model_n)

Overwriting ../../lendingclub/modeling/08_train.py


In [6]:
m = train_model(model_n, X_train, y_train)

/home/justin/anaconda3/envs/lendingclub/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
export_models(m, model_n)
export_data_processing(proc_arti, model_n)

# copied code to incorporate into train script

In [20]:
import pandas as pd
import os
from lendingclub import config
import pickle
dpath = config.data_dir

In [9]:
# from lendingclub.lc_utils import gen_datasets
from j_utils import munging as mg
from sklearn.model_selection import train_test_split

In [17]:
ls {dpath}

base_loan_info.fth                     dev_ids.pkl
bootstrap_test_eval_loan_info_ids.pkl  eval_loan_info.fth
bootstrap_test_idx.pkl                 eval_loan_info_scored.fth
clean_loan_info_api_name_matched.fth   raw_loan_info.fth
clean_loan_info.fth                    scaled_pmt_hist.fth
clean_pmt_history.fth                  strings_loan_info.fth
csvs/                                  train_test_ids.pkl


In [30]:
base_loan_info = pd.read_feather(os.path.join(dpath, 'base_loan_info.fth'))
eval_loan_info = pd.read_feather(os.path.join(dpath, 'eval_loan_info.fth'))
with open(os.path.join(dpath, 'train_test_ids.pkl'), 'rb') as f:
    train_test_ids = pickle.load(f)
    
use_ids = train_test_ids['train']

print(base_loan_info.shape, eval_loan_info.shape, len(use_ids))

tv_base_loan_info = base_loan_info.query('id in @use_ids')
tv_eval_loan_info = eval_loan_info.query('id in @use_ids')

(2507335, 90) (2507335, 45) 1447630


In [ ]:
print

In [16]:
tt_base_loan_info = pd.read_feather(os.path.join(dpath, 'train_testable_base_loan_info.fth'))
tt_eval_loan_info = pd.read_feather(os.path.join(dpath, 'train_testable_eval_loan_info.fth'))
print(tt_base_loan_info.shape, tt_eval_loan_info.shape)

(12434, 90) (12434, 44)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_loan_info, train_eval_loan_info['target_strict'], test_size=.05)

In [ ]:
# fastai style processing
X_train, all_train_colnames, max_dict, min_dict, new_null_colnames, fill_dict, cats_dict, norm_dict = mg.train_proc(X_train)
X_valid = mg.val_test_proc(X_valid, all_train_colnames, max_dict, min_dict, fill_dict, cats_dict, norm_dict)